# Actividad de investigación sobre JAX

Enrique Moreno Alcántara

Incluye:
- Qué es JAX y características principales.
- Comparación con TensorFlow y PyTorch.
- Ecosistema alrededor de JAX.
- Ejemplo práctico completo (optimización + autodiferenciación + `jit` + `vmap`).

## 1. ¿Qué es JAX? y características principales.

JAX es una librería para computación numérica en Python (inspirada en NumPy) que destaca por:

- **API tipo NumPy** (`jax.numpy`): código parecido a NumPy.
- **Autodiferenciación** (reverse/forward-mode) mediante `jax.grad`, `jax.jacfwd`, `jax.jacrev`.
- **Compilación con XLA** mediante `jax.jit` para acelerar operaciones.
- **Vectorización automática** con `jax.vmap` (evita bucles Python).
- **Paralelización** (multi-dispositivo) con `pmap` y `pjit`.


### 1.1. jax.numpy


In [ ]:
import jax
import jax.numpy as jnp

x = jnp.array([1.0, 2.0, 3.0])
y = jnp.sin(x) + x**2
x, y

### 1.2. Autodiferenciación con `grad`

In [ ]:
def f(theta):
    return jnp.sum(jnp.sin(theta) + theta**2)

grad_f = jax.grad(f)
theta = jnp.array([0.1, 0.2, 0.3])
print('f(theta)=', f(theta))
print('grad f(theta)=', grad_f(theta))

### 1.3. `jit`: compilar para acelerar

`jax.jit` compila la función con XLA. Suele dar gran mejora cuando hay operaciones repetidas.
La primera llamada compila (puede tardar más); las siguientes suelen ser mucho más rápidas.


In [ ]:
import time

def heavy(theta, n=2000):
    # ejemplo artificial: muchas operaciones
    z = theta
    for _ in range(n):
        z = jnp.tanh(z) + 0.01 * z
    return jnp.sum(z)

heavy_jit = jax.jit(heavy)

theta = jnp.ones((2000,))

t0 = time.time()
out1 = heavy(theta)          # sin jit
t1 = time.time()

t2 = time.time()
out2 = heavy_jit(theta)      # primera llamada: compila
out2.block_until_ready()
t3 = time.time()

t4 = time.time()
out3 = heavy_jit(theta)      # segunda llamada: ya compilado
out3.block_until_ready()
t5 = time.time()

print('sin jit:', round(t1-t0, 4), 's')
print('jit 1a:', round(t3-t2, 4), 's (incluye compilación)')
print('jit 2a:', round(t5-t4, 4), 's')
print('outputs iguales?', float(out1) == float(out3))

### 1.4. `vmap`: vectorización automática

`vmap` permite aplicar una función sobre un batch sin escribir bucles Python.


In [ ]:
def g(a, b):
    return jnp.sum(a * b)

# Queremos g para muchos pares (a_i, b_i)
A = jnp.arange(12.0).reshape(4, 3)
B = jnp.ones((4, 3))

g_batched = jax.vmap(g, in_axes=(0, 0))
print(g_batched(A, B))

## 2. Comparación: JAX vs TensorFlow vs PyTorch

### 2.1. Estilo de programación
- **JAX**: muy funcional; el rendimiento viene de `jit` + XLA; transformaciones (`grad`, `vmap`, `pmap`).
- **PyTorch**: imperativo (eager) por defecto; excelente ergonomía para investigación; `torch.compile` intenta cerrar la brecha de compilación.
- **TensorFlow**: tuvo histórico más orientado a gráficos; hoy es mayoritariamente eager con `tf.function` para compilar.

### 2.2. Diferenciación automática
- JAX: autodiff integrada, composable y con herramientas para jacobianos/hessianos.
- PyTorch/TF: autodiff muy madura; ecosistemas grandes.

### 2.3. Rendimiento y despliegue
- JAX: muy fuerte en aceleradores y compilación XLA; popular en investigación a gran escala.
- TF: ecosistema de producción amplio (TF Serving, TFLite).
- PyTorch: fuerte en investigación; despliegue con TorchServe / exportadores; cada vez más completo.


## 3. Ecosistema alrededor de JAX

Algunas librerías populares:

- **Flax**: redes neuronales estilo PyTorch/TF (muy usada en la comunidad JAX).
- **Haiku** (DeepMind): alternativa para definir modelos.
- **Optax**: optimizadores (Adam, SGD, etc.).
- **Equinox**: enfoque *PyTorch-like* pero con transformaciones JAX.
- **Diffrax**: ODE solvers diferenciables.
- **BlackJAX**: MCMC / inferencia bayesiana.

Integraciones típicas:
- **XLA** (compilador), **CUDA**/**ROCm** (GPU), **TPU**.
- Pipelines en Python con NumPy/pandas; visualización con matplotlib.


## 4. Ejemplo práctico: regresión lineal con grad + jit + vmap

Vamos a implementar regresión lineal por descenso de gradiente en JAX.
Objetivo: minimizar MSE sobre datos sintéticos.


In [ ]:
import jax
import jax.numpy as jnp

key = jax.random.key(0)

# Datos sintéticos: y = 3x + 2 + ruido
n = 200
x = jax.random.normal(key, (n, 1))
true_w = jnp.array([[3.0]])
true_b = jnp.array([2.0])
noise = 0.3 * jax.random.normal(jax.random.key(1), (n, 1))
y = x @ true_w + true_b + noise

x.shape, y.shape

In [ ]:
def predict(params, x):
    w, b = params
    return x @ w + b  # (n,1)

def mse_loss(params, x, y):
    yhat = predict(params, x)
    return jnp.mean((yhat - y) ** 2)

loss_grad = jax.grad(mse_loss)

# inicialización
params = (jax.random.normal(jax.random.key(2), (1,1)) * 0.1,
          jnp.zeros((1,)))

print('loss inicial:', float(mse_loss(params, x, y)))

In [ ]:
@jax.jit
def step(params, x, y, lr=0.1):
    grads = loss_grad(params, x, y)
    new_params = (params[0] - lr * grads[0],
                  params[1] - lr * grads[1])
    return new_params

# Entrenamiento
for epoch in range(200):
    params = step(params, x, y, lr=0.2)

print('loss final:', float(mse_loss(params, x, y)))
print('w aprendido:', params[0].ravel())
print('b aprendido:', params[1].ravel())

### 4.1. Evaluación en batch con `vmap`

A veces queremos evaluar la loss de muchos parámetros (por ejemplo, en búsqueda aleatoria).
`vmap` permite hacerlo de manera limpia.


In [ ]:
# Creamos varios candidatos alrededor de params
keys = jax.random.split(jax.random.key(3), 10)
ws = params[0] + 0.2 * jax.random.normal(keys[0], (10,1,1))
bs = params[1] + 0.2 * jax.random.normal(keys[1], (10,1))

def loss_single(w, b):
    return mse_loss((w,b), x, y)

losses = jax.vmap(loss_single, in_axes=(0,0))(ws, bs)
losses